In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
import streamlit as st
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [12]:
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
groq_api_key=os.getenv('GROQ_API_KEY')
st.title("chat with llama3 demo")

llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

prompt=ChatPromptTemplate.from_template(
"""
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)

In [13]:
embeddings=OpenAIEmbeddings()
loader=PyPDFDirectoryLoader("./data")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=text_splitter.split_documents(docs[:20])
vector_store=FAISS.from_documents(documents=documents,embedding=embeddings)

In [14]:
retriever=vector_store.as_retriever()

In [20]:
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=vector_store.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)
response=retrieval_chain.invoke({'input':"give me some lines about direct procurment by purchase committe"})

In [21]:
response['answer']

'Based on the provided context, here are some lines about direct procurement by purchase committee:\n\n* Direct procurement by purchase committee is a process where the procurement officer or the purchase committee directly procures goods, consultancy or other services from the supplier without inviting tenders or quotations from multiple suppliers.\n\nPlease note that the provided context is limited to the given text and it may not provide the complete or detailed information about the direct procurement process.'

In [27]:
response

{'input': 'give me some lines about direct procurment by purchase committe',
 'context': [Document(page_content='6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services\nhttps://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 20/308', metadata={'source': 'data\\census.pdf', 'page': 19}),
  Document(page_content='6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services\nhttps://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 15/308', metadata={'source': 'data\\census.pdf', 'page': 14}),
  Document(page_content='6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services\nhttps://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 13/308', metadata={'source': 'data\\census.pdf', 'page': 12}),
  Document(page_content='6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services\nhttps://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 14/308', metada

In [30]:
for i, doc in enumerate(response['context']):
    print("PAGE NO:",doc.metadata['page'],"Content is:",doc.page_content)
    print("--------------------------------")

PAGE NO: 19 Content is: 6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services
https://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 20/308
--------------------------------
PAGE NO: 14 Content is: 6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services
https://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 15/308
--------------------------------
PAGE NO: 12 Content is: 6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services
https://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 13/308
--------------------------------
PAGE NO: 13 Content is: 6/5/24, 9:21 AM ORGI Manual for Procurement of Goods, Consultancy & Other Services
https://censusindia.gov.in/ebook/Procurement/mobile/index.html#p=24 14/308
--------------------------------
